In [20]:
import pandas as pd
from semconstmining.mining.extraction.extractionhandler import DeclareExtractor
from semconstmining.parsing.label_parser.nlp_helper import NlpHelper
from pathlib import Path
import os

from semconstmining.config import Config
from semconstmining.main import get_resource_handler

MODEL_COLLECTION = "opal"

conf = Config(Path(os.getcwd()).parents[1].resolve(), MODEL_COLLECTION)

nlp_helper = NlpHelper(conf)
resource_handler = get_resource_handler(conf, nlp_helper)
extractor = DeclareExtractor(config=conf, resource_handler=resource_handler)
dfs = [extractor.discover_plain_control_flow_constraints(t) for t in resource_handler.bpmn_logs.reset_index().itertuples()]
all_constraints = pd.concat(dfs).astype({conf.LEVEL: "category"})

[2023-05-12 11:06:32,754] p56264 {bert_wrapper.py:239} INFO - loading serialized model
[2023-05-12 11:06:33,990] p56264 {bert_wrapper.py:247} INFO - model loaded
[2023-05-12 11:06:34,938] p56264 {keyedvectors.py:1954} INFO - loading projection weights from /Users/i585907/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
[2023-05-12 11:06:45,677] p56264 {utils.py:447} INFO - KeyedVectors lifecycle event {'msg': 'loaded (400000, 50) matrix of type float32 from /Users/i585907/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-05-12T11:06:45.677869', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 18:29:29) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'load_word2vec_format'}
[2023-05-12 11:06:45,724] p56264 {nlp_helper.py:97} INFO - Loaded 0 known embeddings and 124688 known similarities
[2023-05-12 11:06:45,726] p56264 {resource_handler.py:113} INFO - Loading elements from /

In [21]:
from tqdm import tqdm
from semconstmining.mining.aggregation.subsumptionanalyzer import SubsumptionAnalyzer
all_constraints[conf.SUPPORT] = 1
all_constraints[conf.OBJECT] = ""
activity_constraints = all_constraints
#activity_constraints[conf.CONSTRAINT_STR] = activity_constraints[conf.CONSTRAINT_STR].str.replace("|", "")
#activity_constraints[conf.CONSTRAINT_STR] = activity_constraints[conf.CONSTRAINT_STR].str.strip()
#activity_constraints = activity_constraints[[conf.MODEL_ID,conf.CONSTRAINT_STR]]
# Group by 'conf.MODEL_ID' and generate a list of 'conf.CONSTRAINT_STR' elements for each group
grouped = activity_constraints.groupby(conf.MODEL_ID)
dfs = []
for group_id, group in tqdm(grouped):
    subsumption_analyzer = SubsumptionAnalyzer(conf, group)
    subsumption_analyzer.check_refinement()
    subsumption_analyzer.check_subsumption()
    subsumption_analyzer.check_equal()
    dfs.append(subsumption_analyzer.constraints[~subsumption_analyzer.constraints[conf.REDUNDANT]])
refined = pd.concat(dfs)
refined

  0%|                                                                                                                                                                                                                                                                                  | 1/3724 [00:00<06:13,  9.96it/s][2023-05-12 11:07:56,187] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 0 constraints
[2023-05-12 11:07:56,219] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 9 constraints
[2023-05-12 11:07:56,287] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 26 constraints
  0%|▎                                                                                                                                                                                                                                                                                 | 4/3724 [00:00<03:09, 19.58it/s][2023-05-12 11:07:56,306] p56264 {subsumptionanalyzer.

New constraints: 0
New constraints: 0
New constraints: 18
New constraints: 36
New constraints: 0
New constraints: 0
New constraints: 12
New constraints: 60


  0%|▌                                                                                                                                                                                                                                                                                 | 8/3724 [00:00<02:44, 22.63it/s][2023-05-12 11:07:56,670] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 63 constraints
[2023-05-12 11:07:56,759] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 30 constraints
[2023-05-12 11:07:56,855] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 18 constraints
  0%|▊                                                                                                                                                                                                                                                                                | 11/3724 [00:00<04:55, 12.57it/s]

New constraints: 126
New constraints: 30
New constraints: 36
New constraints: 18


[2023-05-12 11:07:56,898] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 9 constraints
[2023-05-12 11:07:56,965] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 18 constraints
  0%|▉                                                                                                                                                                                                                                                                                | 13/3724 [00:00<04:30, 13.72it/s][2023-05-12 11:07:56,977] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 0 constraints
[2023-05-12 11:07:56,983] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 0 constraints
[2023-05-12 11:07:57,002] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 3 constraints
[2023-05-12 11:07:57,010] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 0 constraints
[2023-05-12 11:07:57,028] p56264 {subsumption

...

[2023-05-12 11:11:58,099] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 5 constraints
[2023-05-12 11:11:58,310] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 63 constraints
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 3718/3724 [04:02<00:00, 21.95it/s][2023-05-12 11:11:58,333] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 0 constraints
[2023-05-12 11:11:58,348] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 3 constraints
[2023-05-12 11:11:58,363] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 3 constraints
[2023-05-12 11:11:58,437] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 32 constraints
100%|███████████████████████████████████████

New constraints: 126
New constraints: 0
New constraints: 6
New constraints: 6
New constraints: 36
New constraints: 0


[2023-05-12 11:11:58,582] p56264 {subsumptionanalyzer.py:47} INFO - Checking subsumption for 45 constraints
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3724/3724 [04:02<00:00, 15.35it/s]


New constraints: 60


,index,Level,Object,constraint_string,op_type,dictionary,data_object,template,left_op,right_op,model_id,model_name,support,redundant
obs_id,,,,,,,,,,,,,,
8d66f32c-ce0f-43dc-82dc-0279823b419a,3.0,Activity,,Exactly1[print po] | |,Unary,[],[],Exactly,print po,NaN,000f3dda41654ea4856efb21d51530a1,3.PMS-20-20-40 Communicate order,1,False
5eb7022b-0ac5-4fc2-bb63-a1c1968c7084,0.0,Activity,,Exactly1[scp review supply scenarios] | |,Unary,[],[],Exactly,scp review supply scenarios,NaN,00322a08d9184f18a703e203e7c62d1d,SSC_MNH_03 Reconciliation Review,1,False
6b41b6b2-9645-42a5-8b53-e38dfe8fcf26,12.0,Activity,,Exactly1[maintain commodity curves] | |,Unary,[],[],Exactly,maintain commodity curves,NaN,003545170a564e9e8e27ba466be508ce,2.MFS-20-150-30 Commodity Curve Reporting,1,False
0fe18c9d-9d5b-445e-adc3-bcd0d1b0009a,17.0,Activity,,Exactly1[compare commodity curve] | |,Unary,[],[],Exactly,compare commodity curve,NaN,003545170a564e9e8e27ba466be508ce,2.MFS-20-150-30 Commodity Curve Reporting,1,False
2fb14d73-5259-4755-81c9-9560b901437c,22.0,Activity,,Exactly1[display dcs based commodity curve] | |,Unary,[],[],Exactly,display dcs based commodity curve,NaN,003545170a564e9e8e27ba466be508ce,2.MFS-20-150-30 Commodity Curve Reporting,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a75c00ff-c381-44e1-a69b-124fdeca529c,NaN,Activity,,Alternate Succession[trigger contact person re...,Binary,[],[],Alternate Precedence,trigger contact person replication,create or change customer,ffc79978d7754e96b25f4a0aef068cf9,2TY - 01 - Integration with SAP Commerce Cloud...,1,False
36be0d85-fd8c-4364-9156-e5568c40f3f2,NaN,Activity,,Alternate Succession[trigger contact person re...,Binary,[],[],Alternate Response,trigger contact person replication,link to corresponding unit optional,ffc79978d7754e96b25f4a0aef068cf9,2TY - 01 - Integration with SAP Commerce Cloud...,1,False
496b8704-66b9-4da7-aa10-19fd8ad5829b,NaN,Activity,,Alternate Succession[trigger contact person re...,Binary,[],[],Alternate Precedence,trigger contact person replication,link to corresponding unit optional,ffc79978d7754e96b25f4a0aef068cf9,2TY - 01 - Integration with SAP Commerce Cloud...,1,False


In [22]:
grouped = refined.groupby(conf.MODEL_ID).agg({conf.CONSTRAINT_STR: list})

# Rename the 'conf.CONSTRAINT_STR' column to 'constraints'
grouped = grouped.rename(columns={conf.CONSTRAINT_STR: 'constraints'})

# Reset the index to turn the 'conf.MODEL_ID' column into a regular column
grouped = grouped.reset_index()

In [23]:
grouped.head()

,model_id,constraints
0,000f3dda41654ea4856efb21d51530a1,[Exactly1[print po] | |]
1,00322a08d9184f18a703e203e7c62d1d,[Exactly1[scp review supply scenarios] | |]
2,003545170a564e9e8e27ba466be508ce,"[Exactly1[maintain commodity curves] | |, Exac..."
3,00558e1b151449168830e2494b38c660,"[Absence2[reject sales quotation] | |, Precede..."
4,00612865e72f4fe1936aeb7e23ad1211,[Exactly1[operation confirmation with backflus...


In [24]:
grouped.to_pickle(MODEL_COLLECTION + "model_id_to_constraints.pkl")